In [1]:
%run "../common.ipynb"

In [ ]:
fileName = "http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT id, measured_at, lat, lon  FROM loc WHERE weatherchecked = 0 AND (loc.record_type <> 'active' OR loc.record_type IS NULL) ORDER BY id DESC LIMIT "+ N +"&type=html"
dfOriginal = LoadDataSet(fileName, columns=None, checkForDateTime=False);


In [23]:
import json
import requests
from datetime import datetime,timedelta
import time

#Finds the index of a station that shares the same date
def binary_search(a, dateobj, lo=0, hi=None):
    if hi is None:
        hi = len(a)
    while lo < hi:
        mid = (lo+hi)//2
        middatetime = a[mid][1]+' '+a[mid][2]
        middateobj = datetime.strptime(middatetime,'%d/%m/%Y %H:%M') 
        
        
        
        #print(middateobj)
        datedifference = middateobj-dateobj       
        negative  = datedifference.days<0
        hourdiff = abs(datedifference.days)*24+datedifference.seconds/3600
        if negative:
            hourdiff = hourdiff*(-1)
            
        #print(hourdiff)
        
        if hourdiff < -4:
            lo = mid+1
        elif hourdiff > 4: 
            hi = mid
        else:
            return mid
    return -1

#Given the index of a good station it finds all the stations within X hours of the loc measured_at datetime
def stationIndexWithinXhours(stations, dateobj, idx,  X):
    goodIDX = [idx]
    
    testidxless = idx - 1
    while testidxless > 0:
        testdatetime = stations[textidxless][1]+' '+stations[testidxless][2]
        testdateobj = datetime.strptime(testdatetime,'%d/%m/%Y %H:%M') 
        testdatedifference = testdateobj - dateobj
        hourdiff = abs(testdatedifference.days)*24+testdatedifference.seconds/3600
        if hourdiff < X:
            goodIDX.append(testidxless)
            testidxless = testidxless-1
        else:
            testidxless = -1
        
    testidxmore = idx + 1
    while testidxmore > 0 and testidxmore < len(stations):
        testdatetime = stations[textidxmore][1]+' '+stations[testidxmore][2]
        testdateobj = datetime.strptime(testdatetime,'%d/%m/%Y %H:%M') 
        testdatedifference = testdateobj - dateobj
        hourdiff = abs(testdatedifference.days)*24+testdatedifference.seconds/3600
        if hourdiff < X:
            goodIDX.append(testidxmore)
            testidxmore = testidxmore+1
        else:
            testidxmore = -1
            
    return goodIDX
        
#Updates the loc_weather table  and loc table to say weather was checked, but no good weather data was found
def updateNoGoodStations(locid , lat, lon, measured_at):
    locid = str(locid)
    lat = str(lat)
    lon = str(lon)
    measured_at = str(measured_at)
    requests.get("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=INSERT INTO loc_weather (id, lat, lon, measured_at) VALUES ("+locid +","+ lat+","+ lon+", TIMESTAMP '"+ measured_at +"')&type=html")
    requests.get("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=UPDATE loc SET weatherchecked=1 WHERE id="+locid +"&type=html")

#finds the scores of listed stations indexs to test. Scores are +1 for each hour away and +1 for every 25 miles away. So lower scores are more accurate
def findScores(stationidxtotest, stations, dateobj, lat, lon):
    scores = []
    for k in stationidxtotest:
        testdatetime = stations[k][1]+' '+stations[k][2]
        testdateobj = datetime.strptime(testdatetime,'%d/%m/%Y %H:%M') 
        testdatedifference = testdateobj - dateobj
        hourdiff = abs(testdatedifference.days)*24+testdatedifference.seconds/3600
    
        stationlat = stations[k][10]
        stationlon = stations[k][11]
        
        milediff = sqrt(69.1*(stationlat-lat)*69.1*(stationlat-lat)+(69.1*(stationlon - lon)*cos(lat/57.3))*(69.1*(stationlon - lon)*cos(lat/57.3)))
        scores.append(milediff/25+hourdiff)
        
    return scores

#Updates loc_weather with the best available information when a station was found to be good enough
def updateBestStation(locid , lat, lon, measured_at, pm10, no2, o3, co, so2, pm2_5, temperature, humidity, weather_lat, weather_lon, weather_recorddate, weather_recordtime, weather_stationid, score):
    locid = str(locid) 
    lat = str(lat)
    lon = str(lon)
    measured_at = str(measured_at)
    pm10 = str(pm10)
    no2 = str(no2)
    o3 = str(o3)
    co = str(co)
    so2 = str(so2)
    pm2_5 = str(pm2_5)
    temperature = str(temperature)
    humidity = str(humidity)
    weather_lat = str(weather_lat)
    weather_lon = str(weather_lon)
    weather_recorddate = str(weather_recorddate)
    weather_recordtime = str(weather_recordtime)
    weather_stationid = str(weather_stationid)
    score = str(score)
    
    requests.get("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=INSERT INTO loc_weather (id, lat, lon, measured_at, pm10, no2, o3, co, so2, pm2_5, temperature, humidity, weather_lat, weather_lon, weather_recorddate, weather_recordtime, weather_stationid, score) VALUES ("+locid +","+ lat+","+ lon+", TIMESTAMP '"+ measured_at +"'"+ pm10+","+ no2+","+ o3+","+ co+","+ so2+","+ pm2_5+","+ temperature+","+ humidity+","+ weather_lat+","+ weather_lon+","+ weather_recorddate+","+ weather_recordtime+","+ weather_stationid+","+ score+")&type=html")
    requests.get("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=UPDATE loc SET weatherchecked=1 WHERE id="+locid +"&type=html")

#Load the loc data from DB
jsonweatherData = json.loads(requests.get("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT *  FROM uaestations ORDER BY to_date(recorddate, 'DD/MM/YYYY') , recordtime::time&type=html").text.split('$rs=')[1])
    

In [31]:
N = 1
N = str(N)
jsonlocData = json.loads(requests.get("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT id, measured_at, lat, lon  FROM loc WHERE weatherchecked = 0 AND (loc.record_type <> 'active' OR loc.record_type IS NULL) ORDER BY id DESC LIMIT "+ N +"&type=html").text.split('$rs=')[1])
#Accessing jsonlocData
#jsonlocData['rows'][i][0] is id
#jsonlocData['rows'][i][1] is measured_at
#jsonlocData['rows'][i][2] is lat
#jsonlocData['rows'][i][3] is lon
#


for i in range(len(jsonlocData['rows'])):
    #Prints the current loc row being checked
    #print(jsonlocData['rows'][i])
    
    locdatetime = jsonlocData['rows'][i][1].split(' ')
    locdate = locdatetime[0]
    locdatesplit = locdate.split('-')
    locyear = locdatesplit[0]
    locmonth = locdatesplit[1]    
    locday = locdatesplit[2]

    
    loctime = locdatetime[1]
    loctimehourminsec = loctime.split('.')[0]   
    
    locdateobj = datetime.strptime(jsonlocData['rows'][i][1].split('.')[0],'%Y-%m-%d %H:%M:%S')   
    
    loctimesplit = loctimehourminsec.split(':')
    lochour = loctimesplit[0]
    locmin = loctimesplit[1]
    locsec = loctimesplit[2]
    
    
    
    loclat = jsonlocData['rows'][i][2]
    loclon = jsonlocData['rows'][i][3]
    
    #Find which entry for weather stations has same date and time within 4 hours as loc entry
    idxOfGoodStationTime = binary_search(jsonweatherData['rows'], locdateobj, lo=0, hi=None)
    
    
    if idxOfGoodStationTime < 0:
        updateNoGoodStations(jsonlocData['rows'][i][0] , jsonlocData['rows'][i][2], jsonlocData['rows'][i][3], jsonlocData['rows'][i][1])
        continue
    
    #Find All Stations indexs within 4 hours
    allGoodTimeIDXs = stationIndexWithinXhours(jsonweatherData['rows'], locdateobj, idxOfGoodStationTime,  4)
    
    scores = findScores(allGoodTimeIDXs, jsonweatherData['rows'], locdateobj, loclat, loclon)
    
    bestscoreidx = scores.index(min(scores))
    
    #print(scores[bestscoreidx])
    #jsonweatherData['rows'][allGoodTimeIDXs[bestscoreidx]] is the best scoring weather station
    if scores[bestscoreidx] < 8:
        updateBestStation(jsonlocData['rows'][i][0] , jsonlocData['rows'][i][2], jsonlocData['rows'][i][3], jsonlocData['rows'][i][1], jsonweatherData['rows'][allGoodTimeIDXs[bestscoreidx]][3], jsonweatherData['rows'][allGoodTimeIDXs[bestscoreidx]][4], jsonweatherData['rows'][allGoodTimeIDXs[bestscoreidx]][5], jsonweatherData['rows'][allGoodTimeIDXs[bestscoreidx]][6], jsonweatherData['rows'][allGoodTimeIDXs[bestscoreidx]][7], jsonweatherData['rows'][allGoodTimeIDXs[bestscoreidx]][8], jsonweatherData['rows'][allGoodTimeIDXs[bestscoreidx]][9], jsonweatherData['rows'][allGoodTimeIDXs[bestscoreidx]][12], jsonweatherData['rows'][allGoodTimeIDXs[bestscoreidx]][10], jsonweatherData['rows'][allGoodTimeIDXs[bestscoreidx]][11], jsonweatherData['rows'][allGoodTimeIDXs[bestscoreidx]][1], jsonweatherData['rows'][allGoodTimeIDXs[bestscoreidx]][2], jsonweatherData['rows'][allGoodTimeIDXs[bestscoreidx]][0], scores[bestscoreidx])
        continue
    else:
        updateNoGoodStations(jsonlocData['rows'][i][0] , jsonlocData['rows'][i][2], jsonlocData['rows'][i][3], jsonlocData['rows'][i][1])
        continue
    
    
    
requests.get("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?cmd=reload")


<Response [200]>